In [2]:
import torch
a=[13.5,12.5,11.5]
a=torch.tensor(a)
a.round()



tensor([14., 12., 12.])

In [2]:
torch.tensor([-2.5000 , -1.5000,   -0.5000,    0.5000 ,   1.5000  ,  2.5000]).round()

tensor([-2., -2., -0.,  0.,  2.,  2.])

In [3]:
X=torch.tensor([[[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4]]])
Y=torch.tensor([1])

X*Y

tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4]]])

In [10]:
import torch
x =torch.tensor([[[1,2,3,4]]])
x.shape
x.expand(1,4,4)
for i in x[0,0,:]:
    print(i)

tensor(1)
tensor(2)
tensor(3)
tensor(4)


In [1]:
out_scale=torch.tensor(0.234)
in_scale=torch.rand(1,1,384)
channel_nums=384
SCALE=torch.rand(1,1,384)
x_q=torch.rand(16,197,384)
x_q_sum=x_q.sum(dim=-1)
std_x_q =  torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)
print(x_q_sum.shape)
for i in range(x_q.shape[2]):#x_q:[1,197,384]
    x_q[:,:,i]=(x_q[:,:,i]*channel_nums-x_q_sum)/std_x_q
    
for i in range(x_q.shape[1]):
    x_q[:,i,:]=x_q[:,i,:]*in_scale

beta=torch.rand(384)



X_q=torch.rand(1,197,384)
print(X_q.shape)
X_Mean=X_q.sum(dim=-1).unsqueeze(-1)*torch.ones(384)
print(X_Mean.shape,X_Mean)


Bias=torch.rand(384)
Bias=Bias.reshape(1,-1,1)
print(Bias.shape)
Bias=Bias*torch.ones(197)
print(Bias.transpose(-1,-2).shape,Bias)

NameError: name 'torch' is not defined

In [3]:
import torch
batch=16
x_q=torch.rand(batch,197,384)#构造输入
in_scale=torch.rand(1,1,384)
out_scale=torch.rand(1)
channel_nums=x_q.shape[-1]#获取通道数
print(x_q[0,:,:])

x_q=(x_q/in_scale).round()
M1=x_q.sum(dim=-1)
print(M1.shape)
print(M1[:,1].shape)
print(M1[:,1].reshape(16,-1,1).shape)
#然后让每一行的点减去对应行的sum
std_x_q = torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)#[1,197]
for i in range(M1.shape[-1]):#对于每一行来说，都是[16,384]维，对于M1的每一行来说，都是[16]维
    x_q[:,i,:]=(x_q[:,i,:]-M1[:,i].reshape(batch,-1))/std_x_q[:,i].reshape(batch,-1)


Gama=torch.rand(1,384)
Beta=torch.rand(1,384)
for i in range(x_q.shape[1]):#对于每一行来说
    x_q[:,i,:]=(x_q[:,i,:]*Gama+Beta).round()
# x=torch.tensor([[[1,2,3],[4,5,6]]])
# M1=x.sum(dim=-1)
# print(x)
# print(M1)
# print(x[:,1,:]-M1[:,1])
a=torch.pow(2,torch.tensor(32))
topk=(1,)
print(topk)


print(Gama.dtype)

tensor([[0.8284, 0.8615, 0.9001,  ..., 0.4303, 0.8895, 0.7683],
        [0.1498, 0.9806, 0.2665,  ..., 0.1643, 0.7077, 0.8185],
        [0.8216, 0.4132, 0.8983,  ..., 0.3648, 0.8573, 0.3017],
        ...,
        [0.5421, 0.3475, 0.2392,  ..., 0.4069, 0.5456, 0.9214],
        [0.9650, 0.2893, 0.0048,  ..., 0.8635, 0.5091, 0.3144],
        [0.7948, 0.3334, 0.4086,  ..., 0.1863, 0.5976, 0.7749]])
torch.Size([16, 197])
torch.Size([16])
torch.Size([16, 1, 1])
(1,)
torch.float32


In [6]:
import torch
#研究acuracy是如何计算的
output=torch.rand(16,1000)#16个batch，1000个类的概率
topk=(1,5)
maxk=max(topk)
_, pred = output.topk(maxk, 1, True, True)

In [48]:
import torch
out_scale=torch.tensor(0.0234)
Gama=((torch.rand(1,1,384)/out_scale*torch.pow(2, torch.tensor(32))).round()/torch.pow(2, torch.tensor(32))).round()
Gama
Beta=((torch.rand(1,1,384)/out_scale*torch.pow(2, torch.tensor(32))).round()/torch.pow(2, torch.tensor(32))).round()
with open ('Scale_Bias.txt','a') as ff:
    for i in range(Gama.shape[-1]):
        ff.write('%02x%02x'%(int(Gama[0,0,i].item())&0xff,int(Beta[0,0,i].item())&0xff))
        ff.write("\n")
ff.close()



In [27]:
a=torch.tensor(100.1).item()
int(a)
a.type()

AttributeError: 'float' object has no attribute 'type'

In [49]:
num=10
b=-12
print('%02x%02x'%(num,b))

0a-c


In [55]:
for x in range(-4,5):
    print ("dec:%d hex:%02X" % (x, x & 0xff))

dec:-4 hex:FC
dec:-3 hex:FD
dec:-2 hex:FE
dec:-1 hex:FF
dec:0 hex:00
dec:1 hex:01
dec:2 hex:02
dec:3 hex:03
dec:4 hex:04
